Importing all the libraries that are needed. 

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

Creating a summy dataset. We are craeting dummy sequence of integers. 

In [84]:
class DummyDataset(Dataset):
    def __init__(self, num_samples, seq_length, vocab_size):
        self.data = torch.randint(0, vocab_size, (num_samples, seq_length))
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

In [85]:
sample_size = 10000
seq_length = 20
vocab_size = 100 
batch_size = 32

In [86]:
dataset = DummyDataset(sample_size, seq_length, vocab_size)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [88]:
print(dataset[0])

tensor([18, 38, 75, 46, 38, 50, 93,  9, 53, 54, 90, 14, 15, 92, 20,  8, 63, 30,
        97, 89])


In [17]:
for i in dataloader:
    print(i.shape)
    print(i[0])
    break #just once

torch.Size([32, 20])
tensor([97, 28, 63, 71, 73, 81, 38, 99, 28, 14, 16, 49, 35, 83, 60, 17, 58, 48,
        34, 28])


We need to create an embedding layer. Essentially we are taking positions or positional indeces as inputs. 
We will get corresponding embeddings which is input + positional sequence as an output 

In [18]:
embedding_dim = 64

In [20]:
embedding = nn.Embedding(vocab_size, embedding_dim)

In [21]:
embedding

Embedding(100, 64)

In [23]:
position_embeddings = nn.Embedding(seq_length, embedding_dim)
position_embeddings

Embedding(20, 64)

In [24]:
def LearnablePositionalEncoding(seq_length, embedding_dim,positions):
    position_embeddings = nn.Embedding(seq_length, embedding_dim)
    return position_embeddings(positions)

In the "for loop" here, we are creating multiple linear encoder layers and a final output layer.We will pass positions and input sequences as inputs, pass them through layers, embed and get the final predictions. 

In [55]:
def ModelWithPositionalEncoding(vocab_size, embedding_dim, seq_length, num_layers,positions):
    embedding = nn.Embedding(vocab_size, embedding_dim)
    positional_encoding = LearnablePositionalEncoding(seq_length, embedding_dim,positions)
    encoder_layers = nn.ModuleList([nn.Linear(embedding_dim, embedding_dim) for _ in range(num_layers)])
    output_layer = nn.Linear(embedding_dim, vocab_size)
    embedded = embedding(inputs) + positional_encoding
    for layer in encoder_layers:
        embedded = layer(embedded)
    output = output_layer(embedded)
    return output

In [56]:
num_layers = 2
batch_size = 32

In [65]:
optimizer = optim.Adam(model.parameters(), lr=0.01)

Here we are creating batches of data in the second loop, in each iteration, we do compute predictions using the previous function and the end of each iteration, we get the loss at the end of each iteration printed on here. 

In [91]:
for i in range(5):
    loss = 0
    for j in dataloader:
        inputs = batch[:, :-1] #exclude last one
        targets = batch[:, 1:] #prediction
        positions = torch.arange(seq_length - 1)  #to length - 2 
        optimizer.zero_grad()
        criterion = nn.CrossEntropyLoss()
        outputs = ModelWithPositionalEncoding(vocab_size, embedding_dim, seq_length, num_layers,positions)
        loss = criterion(outputs.view(-1, vocab_size), targets.reshape(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"i {i+1}/{10}, Loss: {total_loss / len(dataloader)}")

i 1/10, Loss: 98.55494169229135
i 2/10, Loss: 103.2007722991724
i 3/10, Loss: 107.84680554204094
i 4/10, Loss: 112.49128167697796
i 5/10, Loss: 117.13768479009025
